In [ ]:
from chat_solution.create_db import create_db

db = create_db()

In [ ]:
print(db.retrieve("How do you pick a green?"))

In [ ]:
# User input and response handling
from chat_solution.rag import QuizRag

query1 = "what is up?"
query2 = "How do you pick a green?"
rag = QuizRag()  
response = rag.query(query2)
print(response)

In [32]:
# Convert each text chunk to a LangChain Document
from langchain.schema import Document
from chat_solution.create_db import create_text_chunks_from_workshop_data

text_chunks = create_text_chunks_from_workshop_data()
langchain_docs = [
    Document(page_content=text, metadata={"source": f"chunk_{i+1}"})
    for i, text in enumerate(text_chunks)
]

print(len(langchain_docs))
# Display documents with metadata
#for doc in langchain_docs:
#    print(doc.page_content, doc.metadata)

Created 6 chunks of size 700 with overlap 200
6


In [28]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from langchain_mistralai import ChatMistralAI
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatMistralAI(model="mistral-large-latest"))
metrics = [LLMContextRecall(), FactualCorrectness(), Faithfulness()]


In [24]:
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

generator_llm = LangchainLLMWrapper(ChatMistralAI(model="mistral-large-latest"))
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
generator_embeddings = LangchainEmbeddingsWrapper(embedding_model)

In [31]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm)
dataset = generator.generate_with_langchain_docs(langchain_docs,transforms_embedding_model=generator_embeddings, testset_size=10)



Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:  50%|█████     | 6/12 [00:43<00:34,  5.79s/it]Property 'themes' already exists in node '2eb6d9'. Skipping!
Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:  58%|█████▊    | 7/12 [00:44<00:22,  4.49s/it]Property 'themes' already exists in node '8af9d7'. Skipping!
Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:  67%|██████▋   | 8/12 [00:48<00:17,  4.33s/it]Property 'themes' already exists in node '3f5c71'. Skipping!
Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:  75%|███████▌  | 9/12 [01:03<00:23,  7.69s/it]

In [ ]:
## Prepared questions dataset from huggingface
from datasets import load_dataset
dataset = load_dataset("atitaarora/food_lab_green_qna", split="train")
len(dataset)

In [ ]:
## Sample question data
dataset[2]

In [ ]:
#from ragas import evaluate
#results = evaluate(dataset=dataset, metrics=metrics, llm=evaluator_llm)

In [155]:
## Preparation of Eval dataset for RAGAS (https://docs.ragas.io/en/stable/concepts/components/eval_sample/?h=singleturnsample#example)
##for ragas dataset needs to be in the designated format 
from ragas import EvaluationDataset, SingleTurnSample
from ragas.metrics import Faithfulness
from datasets import load_dataset
from ragas import evaluate
import time

samples = []
eval_size = 5

for i in range(eval_size):
    entry = dataset[i]
    
    # Perform the query with a delay to limit to 1 request per second
    user_query = entry['query']
    response = rag.query(user_query)
    
    sample = SingleTurnSample(
        user_input=user_query,
        reference=entry['reference_answer'],
        response=response,
        retrieved_contexts=db.retrieve(user_query),
    )
    samples.append(sample)
    
    # Wait for 1-2 second before proceeding to the next iteration as we are limited by Mistral API
    time.sleep(2)


In [ ]:
import pandas as pd
df = pd.DataFrame(samples)

# Display the DataFrame as a table
print(df)

In [ ]:
## Actual Evaluation
from ragas.metrics import LLMContextPrecisionWithReference
from ragas.metrics import NonLLMContextRecall
from ragas.metrics import LLMContextRecall
from ragas.metrics import Faithfulness
from ragas.metrics import ResponseRelevancy

eval_dataset = EvaluationDataset(samples=samples)

faithfulness = Faithfulness()
context_precision = LLMContextPrecisionWithReference()
context_recall = NonLLMContextRecall()
llm_context_recall = LLMContextRecall()
answer_relevancy = ResponseRelevancy()

eval_results = evaluate(
        dataset=eval_dataset,
        metrics=[
                faithfulness,
                answer_relevancy,
                #context_recall, #This metric [non_llm_context_recall] that is used requires the following additional columns ['reference_contexts'] to be present in the dataset.
                llm_context_recall,
                context_precision,
        ],
       #llm=evaluator_llm
       raise_exceptions=False 
    )
#eval_results = evaluate(
#    dataset=eval_dataset,
#    metrics=[metric],
#llm=evaluator_llm
#)

In [ ]:
evaluation_result_df = eval_results.to_pandas()
evaluation_result_df.iloc[:5]